# Image Processing SS 16 - Assignment - 04

### Deadline is 1.6.2016 at 16:00 o'clock

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [1]:
# display the plots inside the notebook
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from skimage.data import astronaut
from skimage.color import rgb2gray

import random
try:
    from StringIO import StringIO as BytesIO
except ImportError:
    from io import BytesIO
    
from PIL import Image
import itertools

pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

# Exercise 1 - Filters - 5 Points

Qualify the noise and sharpness in the images. Make a plot images, noise

In [3]:
img = rgb2gray(astronaut() / 255.)
plt.imshow(img, cmap='gray')
plt.show()
img.shape

In [4]:
def salt_peper(img, p):
    peper_mask = np.random.binomial(1, p, img.shape).astype(np.bool)
    salt_mask = np.random.binomial(1, p, img.shape).astype(np.bool)
    img_noise = img.copy()
    img_noise[peper_mask] = 0
    img_noise[salt_mask] = 1
    return img_noise

salt_peper_noises = [0.01, 0.03, 0.1]
imgs_salt_peper = [salt_peper(img, p) for p in salt_peper_noises]
#for img_salt_peper in imgs_salt_peper:
#    plt.imshow(img_salt_peper, cmap='gray')
#    plt.show()

In [5]:
def gaussian_noise(img, std):
    return np.clip(img + np.random.normal(0, std, img.shape), 0, 1)

gaussian_noises = (0.05, 0.10, 0.2)
imgs_gaussian_noise = [gaussian_noise(img, s) for s in gaussian_noises]
#for img_gaussian in imgs_gaussian_noise:
#    plt.imshow(img_gaussian, cmap='gray')
#    plt.show()

In [6]:
# implement the filters:
import scipy.signal

def arithmetic_mean(img):
    d = 1./9.
    arithmetic_kernel = np.array([[d, d, d], [d,d,d], [d,d,d]])
    
    return scipy.signal.convolve2d(img[:,:], arithmetic_kernel, mode='same')

def getNeighbors(img, r, c, Sxy):
    if (Sxy == 9):
        return np.array([[img[r-1,c-1], img[r-1,c], img[r-1,c+1]], [img[r,c-1], img[r,c], img[r,c+1]], [img[r+1,c-1], img[r+1,c], img[r+1,c+1]]])
    else:
        return np.array([img[(r-2):(r+3), (c-2):(c+3)]])

# test for security
t = np.array([[1,2,3],[4,5,6],[7,8,9]])
t1 = getNeighbors(t,1,1,9)
t1[0,0] = 100
if (np.sum(t1) == np.sum(t)):
    raise
    
def geometric_mean(img):
    res = np.copy(img)
    for row in range(1,res.shape[0]-1):
        for column in range(1,res.shape[1]-1):
            neighbors = getNeighbors(img, row, column, 9)
            res[row, column] = np.sqrt(float(np.prod(neighbors)))
    return res / np.max(res)

def contraharmoic_mean(img, q=1): # Like in wikipedia we choose q = 1
    res = np.copy(img)
    for row in range(1,res.shape[0]-1):
        for column in range(1,res.shape[1]-1):
            neighbors = getNeighbors(img, row, column, 9)
            counter = np.sum(np.power(neighbors, q+1)) / 9.0
            denominator = np.sum(np.power(neighbors, q)) / 9.0
            if denominator == 0 or denominator == np.inf or np.isnan(denominator):
                denominator = 0.01
                #print(denominator, row, column)
                #raise Exception("")
            res[row,column] = counter / float(denominator)
    return np.clip(res,0,1)

def realMedian(m,d): 
    m1 = np.reshape(m,d)
    m1 = np.sort(m1)
    return m1[int(m1.size/2)]

def median_filter(img):
    result = np.copy(img)
    
    for row in range(1,result.shape[0]-1):
        for column in range(1,result.shape[1]-1):
            neighbors = getNeighbors(img, row, column, 9)
            result[row,column] = realMedian(neighbors, 9)
            
    return result
    

def adaptive_mean(img):
    res = np.copy(img)
    Smax = 25
    
    for row in range(3,res.shape[0]-3):
        for column in range(3,res.shape[1]-3):
            Sxy = 9
            
            while (True):
                neighbors = getNeighbors(img, row, column, Sxy)

                Zmean = np.mean(neighbors)
                Zmin = np.min(neighbors)
                Zmax = np.max(neighbors)
                Zxy = img[row,column]

                A1 = Zmean - Zmin
                A2 = Zmean - Zmax

                if (A1>0 and A2<0):
                    #doB
                    B1 = Zxy - Zmin
                    B2 = Zxy - Zmax
                    
                    if (B1 > 0 and B2 < 0):
                        res[row,column] = Zxy
                        break
                    else:
                        res[row,column] = Zmean
                        break
                    
                elif (Sxy < Smax):
                    Sxy = Smax
                else:
                    res[row, column] = Zmean
                    break
            
    return np.clip(res,0,1)

def adaptive_median(img):
    res = np.copy(img)
    Smax = 25
    
    for row in range(3,res.shape[0]-3):
        for column in range(3,res.shape[1]-3):
            Sxy = 9
            
            while (True):
                neighbors = getNeighbors(img, row, column, Sxy)

                Zmed = realMedian(neighbors,Sxy)
                Zmin = np.min(neighbors)
                Zmax = np.max(neighbors)
                Zxy = img[row,column]

                A1 = Zmed - Zmin
                A2 = Zmed - Zmax

                if (A1>0 and A2<0):
                    #doB
                    B1 = Zxy - Zmin
                    B2 = Zxy - Zmax
                    
                    if (B1 > 0 and B2 < 0):
                        res[row,column] = Zxy
                        break
                    else:
                        res[row,column] = Zmed
                        break
                    
                elif (Sxy < Smax):
                    Sxy = Smax
                else:
                    res[row, column] = Zmed
                    break
                
    return np.clip(res,0,1)

plt.imshow(geometric_mean(imgs_salt_peper[0]), cmap='gray')
plt.show()


In [7]:
filters = [arithmetic_mean, geometric_mean, contraharmoic_mean, median_filter, 
           adaptive_mean, adaptive_median]

for filter in filters:
    for sp, img_sp in zip(salt_peper_noises, imgs_salt_peper):
        print(filter.__name__ + ", salt peper noise: {}".format(sp))
        plt.subplot(221)
        plt.imshow(img_sp, cmap='gray')
        plt.subplot(222)
        plt.imshow(filter(img_sp), cmap='gray')
        plt.show()

In [8]:
for filter in filters:
    for gn, img_gn in zip(gaussian_noises, imgs_gaussian_noise):
        print(filter.__name__ + ", gaussian noise: {}".format(gn))
        plt.subplot(221)
        plt.imshow(img_gn, cmap='gray')
        plt.subplot(222)
        plt.imshow(filter(img_gn), cmap='gray')
        plt.show()

# Exercise 2 - Sharpening - 5 Points

Make the `img_blurry` sharp again. You need to implement the `sharp_laplacian` and `unsharp_masking`. You might
consider some processing step before sharping.

In [9]:
from skimage.filters import gaussian

In [61]:
img_blurry_noise = gaussian_noise(gaussian(img , 1.6), 0.07)


# preprocessing: Remove noise with a gaussian filter
img_blurry = gaussian(img_blurry_noise, 1.8)


plt.subplot(121)
plt.imshow(img_blurry_noise, cmap='gray')
plt.title("Blurred and noisy image")
plt.subplot(122)
plt.imshow(img_blurry, cmap='gray')
plt.title("Noise removed with gaussian filter")
plt.show()

In [62]:
def laplacian(f, x, y):
    return f[x+1,y] + f[x-1,y] + f[x,y+1] + f[x,y-1] - 4*f[x,y]

def sharp_laplacian(img):
    """Performce sharpening with a laplacian filter"""
    result = np.copy(img)
    
    for row in range(1, result.shape[0]-1):
        for column in range(1, result.shape[1]-1):
            result[row,column] = laplacian(img,row,column)
            
    result = img - 2 * result
    
    result = np.clip(result,0,1)
    
            
    return result

def unsharp_masking(img):
    """Performce sharpening by unsharp masking"""
    #original
    plt.subplot(131)
    plt.imshow(img, cmap='gray')
    
    #blur
    plt.subplot(132)
    result = np.copy(img)
    blur = gaussian(img , 1.6)
    plt.imshow(blur, cmap='gray')
    
    #difference
    diff = result - blur
    
    #result
    plt.subplot(133)
    result = np.clip(result + 3*diff, 0,1)
    plt.imshow(blur, cmap='gray')
    plt.show()
    
    
    return result

In [63]:
plt.imshow(sharp_laplacian(img_blurry), cmap='gray')
plt.show()

In [64]:
plt.imshow(unsharp_masking(img_blurry), cmap='gray')
plt.show()